# Predicción empleando nuestro modelo predictivo

**Modelo**: RandomForest

**Variable respuesta**: 'Totales'

**Variables predictoras**: codificadas sin estandarizar.

In [253]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

# Poner descansos en código.
# ------------------------------------------------------------------------------
import time

# Librería para crear archivos pickle.
# ------------------------------------------------------------------------------
import pickle

In [166]:
df = pd.read_csv('data/totales_encod.csv', index_col = 0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,total,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,14.1,18.2,81.0,10.7,985,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,14.9,17.7,70.0,16.7,801,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,8.0,9.5,44.0,16.6,1349,2018-01-03,NaN,0,1,2,1,0,0


In [167]:
df.drop(['registro','fecha_nueva','festividad','sens_termica'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'humedad', 'viento', 'total', 'festividad_1',
       'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo', 'año_map',
       'estacion_map'],
      dtype='object')

In [97]:
X=df.drop(['total'],axis=1)
y=df['total']

In [98]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [16,17], "max_features": [1,2,3], "min_samples_split": [5, 75, 200], "min_samples_leaf": [5,75,200]}

In [100]:
gs_rf = GridSearchCV(estimator=RandomForestRegressor(), param_grid= param, cv=10, verbose=-1,
                        return_train_score = True, scoring="neg_mean_squared_error")

In [101]:
gs_rf.fit(x_train, y_train)
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=16, max_features=3, min_samples_leaf=5,
                      min_samples_split=5)

In [102]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [103]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
df_totales_encod = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest totales codificadas")
df_totales_encod

,MAE,MSE,RMSE,R2,set,modelo
0,452.901207,341945.670063,584.761208,0.900099,test,Random Forest totales codificadas
1,423.474037,345989.230270,588.208492,0.909325,train,Random Forest totales codificadas


## PROBLEMA: falta columna para año 2023

In [229]:
df.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0


In [ ]:
######################## SOLUCIÓNAR PROBLEMA ###########################

In [228]:
#Codificamos de nuevo la columna año_bueno, ya que para poder hacer la prediccion neceitamos una columna con el año.
df2=df.copy()
df2['año_map_23']=0
df2.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map,año_map_23
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0,0


In [251]:
def probar_modelo(dataframe,parametros):
    X3=dataframe.drop(['total'],axis=1)
    y3=dataframe['total']
    time.sleep(3)
    x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=3)
    time.sleep(3)
    gs_rf3 = GridSearchCV(estimator=RandomForestRegressor(), param_grid= parametros, cv=10, verbose=-1,
                        return_train_score = True, scoring="neg_mean_squared_error")
    time.sleep(3)
    gs_rf3.fit(x3_train, y3_train)
    time.sleep(3)
    bosque3 = gs_rf2.best_estimator_
    time.sleep(3)
    y3_pred_test_rf = bosque3.predict(x3_test)
    y3_pred_train_rf = bosque3.predict(x3_train)
    time.sleep(3)
    df_totales_encod3 = metricas(y3_test, y3_train, y3_pred_test_rf, y3_pred_train_rf, "Random Forest totales codificadas")

    return  bosque3,df_totales_encod3

In [224]:
param5 = {"max_depth": [15,16,17], "max_features": [1,2,3], "min_samples_leaf": [2,3,5],"min_samples_split": [3, 5, 10]}
bosque5,metricas5=probar_modelo(df2,param5)

In [225]:
bosque5

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=3)

In [226]:
metricas5

,MAE,MSE,RMSE,R2,set,modelo
0,326.238546,199586.254222,446.750774,0.944760,test,Random Forest totales codificadas
1,342.367171,245731.658964,495.713283,0.934894,train,Random Forest totales codificadas


In [262]:
param4 = {"max_depth": [15], "max_features": [3], "min_samples_leaf": [2],"min_samples_split": [3]}
bosque4,metricas4=probar_modelo(df2,param4)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- año_map_23
Feature names seen at fit time, yet now missing:
- año_map_22


In [269]:
X4=df2.drop(['total'],axis=1)
y4=df2['total']
time.sleep(3)
x4_train, x4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=4)
time.sleep(3)
param4 = {"max_depth": [15], "max_features": [3], "min_samples_leaf": [2],"min_samples_split": [3]}
gs_rf4 = GridSearchCV(estimator=RandomForestRegressor(), param_grid= param4, cv=10, verbose=-1,
                        return_train_score = True, scoring="neg_mean_squared_error")
time.sleep(3)
gs_rf4.fit(x4_train, y4_train)
time.sleep(3)
bosque4 = gs_rf4.best_estimator_
bosque4

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=3)

In [270]:
y4_pred_test_rf = bosque4.predict(x4_test)
y4_pred_train_rf = bosque4.predict(x4_train)
time.sleep(3)
df_totales_encod4 = metricas(y4_test, y4_train, y4_pred_test_rf, y4_pred_train_rf, "Random Forest totales codificadas")
df_totales_encod4

,MAE,MSE,RMSE,R2,set,modelo
0,516.457413,495236.409642,703.730353,0.867590,test,Random Forest totales codificadas
1,293.006734,172767.684296,415.653322,0.953851,train,Random Forest totales codificadas


In [ ]:
#Guardamos nuestro modelo en un pickle para no tener que correr el modelo cada vez que queramos
#  hacer una predicción.
with_open(data/sonia_modelo_rf_totales.pkl, 'wb') as fp:
    pickle.dump(bosque4,fp)

In [ ]:
#Abrimos el pickle creado y aplicamos el modelo almacenado en él.
with_open(data/sonia_modelo_rf_totales.pkl, 'wb') as f:
    forest=pickle.load()

## Predicción

### Estandarización
No tenemos que estandarizar nada ya que en nuestro modelo empleamos los datos sin estandarizar.

### Encondings empleados en el modelo

- "año_bueno" fue codificada haciando la columna "año_map" donde:

In [ ]:
#año_map={2018:0,2019:1}
#df['año_map']=df['año_bueno'].map(año_map)
#df.drop(['año_bueno'],axis=1,inplace=True)

- "estacion_correcta" fue codificada haciendo "estacion_map" donde:

In [ ]:
#estacion_map={'invierno':0,'primavera':1,'otoño':2,'verano':3}
#df['estacion_map']=df['estacion_correcta'].map(estacion_map)
#df.drop(['estacion_correcta'],axis=1,inplace=True)

Por tanto para nuestra prediccion deberemos aplicar:
- **año**: en las predicciones tendremos el año 2023, que se aplicará en la columna creada en los pasos anteriores de este documento "año_map_23"
- **estación**: al recibir los datos con los que predecir, la estación será codificada de la misma manera que en el modelo:

``` python
        estacion_map={'invierno':0,'primavera':1,'otoño':2,'verano':3}
        df['estacion_map']=df['estacion_correcta'].map(estacion_map)
        df.drop(['estacion_correcta'],axis=1,inplace=True)
```

In [230]:
#Prediccion Ana profe
usuario={'season':'spring','holiday':'si','temp':13,'atemp':15.7,'hum':70,'windspeed':10.9,'weekday':'Martes'}
df_usuario=pd.DataFrame(usuario,index=[0])
df_usuario

,season,holiday,temp,atemp,hum,windspeed,weekday
0,spring,si,13,15.7,70,10.9,Martes


In [242]:
#Vemos que el DataFrame que nos da el cliente no tiene las mismas columnas que tenemos
#  nosotras para nuestro modelo.
df2.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map,año_map_23
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0,0


In [243]:
df2['clima'].unique()

array([2, 1, 3])

In [244]:
df2['festividad_1'].unique()

array([1, 0])

In [245]:
df2['mes_bueno'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [246]:
df2['dia_semana_nuevo'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [247]:
df2['no_laboral_nuevo'].unique()

array([0, 1])

In [248]:
df2['año_map'].unique()

array([0, 1])

In [249]:
df2['estacion_map'].unique()

array([0, 1, 3, 2])

In [250]:
df2['año_map_23'].unique()

array([0])

## PRUEBA1

In [271]:
df2.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map,año_map_23
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0,0


In [301]:
df3=df2.copy()

In [302]:
df3['año_map_18']=df3['año_map'].map({0:1,1:0})
df3['año_map_19']=df3['año_map']
df3.drop(['año_map'],axis=1,inplace=True)
df3.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_map,año_map_23,año_map_18,año_map_19
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0,1,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0,1,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0,1,0


In [291]:
def probar_modelo2(dataframe,parametros):
    X3=dataframe.drop(['total'],axis=1)
    y3=dataframe['total']
    time.sleep(3)
    x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=9)
    time.sleep(3)
    gs_rf3 = GridSearchCV(estimator=RandomForestRegressor(), param_grid= parametros, cv=10, verbose=-1,
                        return_train_score = True, scoring="neg_mean_squared_error")
    time.sleep(3)
    gs_rf3.fit(x3_train, y3_train)
    time.sleep(3)
    bosque3 = gs_rf3.best_estimator_
    time.sleep(3)
    y3_pred_test_rf = bosque3.predict(x3_test)
    y3_pred_train_rf = bosque3.predict(x3_train)
    time.sleep(3)
    df_totales_encod3 = metricas(y3_test, y3_train, y3_pred_test_rf, y3_pred_train_rf, "Random Forest totales codificadas")

    return  bosque3,df_totales_encod3

In [282]:
param6 = {"max_depth": [15,16,17], "max_features": [1,2,3], "min_samples_leaf": [2,3,5],"min_samples_split": [3, 5, 10]}
bosque6,metricas6=probar_modelo2(df3,param6)

In [283]:
bosque6

RandomForestRegressor(max_depth=17, max_features=3, min_samples_leaf=2,
                      min_samples_split=3)

In [284]:
metricas6

,MAE,MSE,RMSE,R2,set,modelo
0,488.190445,463067.625994,680.490724,0.871835,test,Random Forest totales codificadas
1,298.295930,188611.608608,434.294380,0.950027,train,Random Forest totales codificadas


In [292]:
param7 = {"max_depth": [15], "max_features": [3], "min_samples_leaf": [2],"min_samples_split": [3]}
bosque7,metricas7=probar_modelo2(df3,param7)

In [293]:
bosque7

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=3)

In [294]:
metricas7

,MAE,MSE,RMSE,R2,set,modelo
0,478.981493,437070.950815,661.113418,0.896789,test,Random Forest totales codificadas
1,300.954040,190060.913374,435.959761,0.947457,train,Random Forest totales codificadas


In [295]:
param8 = {"max_depth": [15], "max_features": [3], "min_samples_leaf": [2],"min_samples_split": [2]}
bosque8,metricas8=probar_modelo2(df3,param8)
metricas8

,MAE,MSE,RMSE,R2,set,modelo
0,480.292477,428415.140801,654.534293,0.898833,test,Random Forest totales codificadas
1,297.776379,185327.199935,430.496458,0.948766,train,Random Forest totales codificadas


In [308]:
param7 = {"max_depth": [14], "max_features": [3], "min_samples_leaf": [2],"min_samples_split": [3]}
bosque7,metricas7=probar_modelo2(df3,param7)
metricas7

,MAE,MSE,RMSE,R2,set,modelo
0,498.777171,465083.385071,681.970223,0.890174,test,Random Forest totales codificadas
1,299.933755,186293.759223,431.617608,0.948499,train,Random Forest totales codificadas


## PRUEBA2

In [296]:
df2.head(3)

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map,año_map_23
0,2,14.1,81.0,10.7,985,1,1,0,0,0,0,0
1,2,14.9,70.0,16.7,801,0,1,1,1,0,0,0
2,1,8.0,44.0,16.6,1349,0,1,2,1,0,0,0


In [300]:
df4=df2.copy()

In [305]:
param10 = {"max_depth": [15,16,17], "max_features": [1,2,3], "min_samples_leaf": [2,3,5],"min_samples_split": [3, 5, 10]}
bosque10,metricas10=probar_modelo2(df3,param10)

In [304]:
metricas10

,MAE,MSE,RMSE,R2,set,modelo
0,516.771395,472753.407985,687.570657,0.888363,test,Random Forest totales codificadas
1,298.737051,178215.905024,422.156257,0.950732,train,Random Forest totales codificadas


In [307]:
bosque10

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=3)

In [306]:
metricas10

,MAE,MSE,RMSE,R2,set,modelo
0,499.520953,459660.972881,677.983018,0.891454,test,Random Forest totales codificadas
1,302.593536,188939.228085,434.671402,0.947767,train,Random Forest totales codificadas
